# Practica final de Estadistica de Bernardo Koron

In [ ]:
library(ggplot2)
library(magrittr)
options(repr.plot.height=8,repr.plot.width=16)

Cargar los datos en un dataframe llamado: airbnb

In [ ]:
airbnb<-read.csv('data//airbnb.csv',sep = ',', stringsAsFactors = F)

Mostrar las primeras 6 filas del dataframe

In [ ]:
head(airbnb)

Renombrar las columnas de la siguiente forma:

| Nombre original | Nuevo nombre |
| - | - |
| Zipcode | CodigoPostal |
| Neighbourhood.Cleansed | Barrio |
| Property.Type	| TipoPropiedad |
| Room.Type | TipoAlquiler |
| Accommodates | MaxOcupantes |
| Bathrooms	| NumBanyos |
| Bedrooms	| NumDormitorios |
| Beds	|  NumCamas |
| Bed.Type	| TipoCama |
| Amenities | Comodidades |
| Square.Feet | PiesCuadrados |
| Price	| Precio |
| Review.Scores.Rating | Puntuacion |

In [ ]:
newnames<-c("CodigoPostal","Barrio","TipoPropiedad","TipoAlquiler","MaxOcupantes","NumBanyos",
"NumDormitorios","NumCamas","TipoCama","Comodidades","PiesCuadrados","Precio","Puntuacion")

In [ ]:
colnames(airbnb) <- newnames

Crea una nueva columna llamada MetrosCuadrados a partir de la columna PiesCuadrados.

Ayuda: 1 pie cuadrado son 0,092903 metros cuadrdados

In [ ]:
airbnb$MetrosCuadrados <- airbnb$PiesCuadrados * 0.092903

Miremos el código postal. Es una variable con entradas erroneas.
Hay valores como '', '-'  y '28' que deberían ser considerados como NA.
Así mismo también debería ser NA todos los que no compiencen por 28, ya que estamos con códigos postales de Madrid

El código postal 28002, 28004 y 28051 tienen entradas repetidas. 
Por ejemplo las entradas 28002\n20882 deberían ir dnetro de 28002

El codigo 2804 debería ser 28004, 2805 deberia ser 28005 y 2815 juncto con 2815 debería ser 28015

Limpia los datos de la columna Codigo Postal

In [ ]:
cpFilter <- function(x) {
    if (is.na(x)) {
        return (NA)
    }
    if (nchar(substr(x,1,5)) == 4 && substr(x,1,2) == 28){
        return(paste0(substr(x,1,2), 0, substr(x,3,4)))
    }
    if (substr(x,1,5)>="28000" && substr(x,1,5)<="28099" && nchar(substr(x,1,5)) == 5){
        return(substr(x,1,5)) 
    }
    return(NA)
}

airbnb$CodigoPostal <- sapply(airbnb$CodigoPostal, cpFilter)

Una vez limpios los datos ¿Cuales son los códigos postales que tenemos? 

In [ ]:
airbnb$CodigoPostal <- as.factor(airbnb$CodigoPostal)
unique(airbnb$CodigoPostal)

¿Cuales son los 5 códigos postales con más entradas? ¿Y con menos? ¿Cuantas entradas tienen?

¿Cuales son los barrios que hay en el código postal 28012?

In [ ]:
airbnb$CodigoPostal %>% table() %>% sort(decreasing = T) %>% head(5)
airbnb$CodigoPostal %>% table() %>% sort(decreasing = F) %>% head(5)

In [ ]:
airbnb[airbnb$CodigoPostal == 28012 & !is.na(airbnb$CodigoPostal) ,"Barrio"] %>% unique()

¿Cuantas entradas hay en cada uno de esos barrios para el codigo postal 28012?

In [ ]:
airbnb[airbnb$CodigoPostal == 28012 & !is.na(airbnb$CodigoPostal) ,"Barrio"] %>%
    table() %>% sort(decreasing = T)
    

¿Cuantos barrios hay en todo el dataset airbnb? ¿Cuales son?

In [ ]:
airbnb$Barrio %>% unique
airbnb$Barrio %>% unique %>% length

¿Cuales son los 5 barrios que tienen mayor número entradas?

In [ ]:
airbnb$Barrio %>% table() %>% sort(decreasing = T) %>% head(5)

¿Cuantos Tipos de Alquiler diferentes hay? ¿Cuales son? ¿Cuantas entradas en el dataframe hay por cada tipo?

In [ ]:
airbnb$TipoAlquiler <- as.factor(airbnb$TipoAlquiler)
airbnb$TipoAlquiler %>% unique()
airbnb$TipoAlquiler %>% unique() %>% length()
airbnb$TipoAlquiler %>% table()

Muestra el diagrama de cajas del precio para cada uno de los diferentes Tipos de Alquiler

In [ ]:
ggplot(data=airbnb, aes(x=TipoAlquiler, y=Precio, color=TipoAlquiler))+
 geom_boxplot()+
 scale_color_discrete(name="Precio")

Cual es el precio medio de alquiler de cada uno, la diferencia que hay ¿es estadísticamente significativa? ¿Con que test lo comprobarías?

In [ ]:
mean_precio_tipo <- aggregate(Precio~TipoAlquiler, airbnb, mean, na.rm=TRUE)
mean_precio_tipo

In [ ]:
# Para comparar los tres grupos, primero debemos comprobar si tienen una distribucion normal o no.
airbnb$Precio[airbnb$TipoAlquiler == "Entire home/apt"] %>% head(5000) %>% shapiro.test()
airbnb$Precio[airbnb$TipoAlquiler == "Private room"] %>% head(5000) %>% shapiro.test()
airbnb$Precio[airbnb$TipoAlquiler == "Shared room"] %>% head(5000) %>% shapiro.test()


In [ ]:
# Como los p-valor de los tres casos son muy bajos, la distribucion no es normal.
# Por lo tanto se utilizara el test de Kruskal Wallis
kruskal.test(Precio ~ TipoAlquiler, data = airbnb)

In [ ]:
# Como el p valor es muy bajo, se puede afirmar que la diferencia que hay en los precios, 
# si es estadisticamente significativa

Filtra el dataframe cuyos tipo de alquiler sea  'Entire home/apt' y guardalo en un dataframe llamado 
*airbnb_entire*.
Estas serán las entradas que tienen un alquiler del piso completo.

In [ ]:
airbnb_entire <- airbnb[airbnb$TipoAlquiler == "Entire home/apt", ]

¿Cuales son los 5 barrios que tienen un mayor número de apartamentos enteros en alquiler?
Nota: Mirar solo en airbnb_entire. A partir de este punto y hasta que se diga lo contrario partiremos de airbnb_entire.

In [ ]:
airbnb_entire$Barrio %>% table() %>% sort(decreasing = T) %>% head(5)

¿Cuales son los 5 barrios que tienen un mayor precio medio de alquiler para apartamentos enteros (es decir, del dataframe airbnb_entire)?

¿Cual es su precio medio?

Ayuda: Usa la función aggregate `aggregate(.~colname,df,mean,na.rm=TRUE)`

In [ ]:
airbnb_entire_agg <- aggregate(Precio~Barrio, airbnb_entire, mean, na.rm=TRUE)

airbnb_entire_agg[order(airbnb_entire_agg$Precio, decreasing = TRUE),] %>% head(5)

¿Cuantos apartamentos hay en cada uno de esos barrios?

Mostrar una dataframe con el nombre del barrio, el precio y el número de entradas.

Ayuda: Podeis crear un nuevo dataframe con las columnas "Barrio" y "Freq" que contenga el número de entradas en cada barrio y hacer un merge con el dataframe del punto anterior.

In [ ]:
# Crea DF con la cantidad de entradas de cada barrio
airbnb_barrio_freq <- airbnb_entire$Barrio %>% table() %>% as.data.frame()
colnames(airbnb_barrio_freq) <- c("Barrio", "Freq")

In [ ]:
# Merge de ambos DF
airbnb_precio_freq <- merge(airbnb_barrio_freq, airbnb_entire_agg, by="Barrio", all=T)
# Ordenasegun precio y se muestra los 5 primeros
airbnb_precio_freq[order(airbnb_precio_freq$Precio, decreasing = T),] %>% head(5)

Partiendo del dataframe anterior, muestra los 5 barrios con mayor precio, pero que tengan más de 100 entradas de alquiler.

In [ ]:
# Barrios con mas de 100 entradas
airbnb_precio_freq_100 <- airbnb_precio_freq[airbnb_precio_freq$Freq > 100, ]

airbnb_5mayor_precio <- airbnb_precio_freq_100[airbnb_precio_freq_100$Precio %>% order(decreasing=T),] %>% head(5)

airbnb_5mayor_precio

Dibuja el diagrama de densidad de distribución de los diferentes precios. Serían 5 gráficas, una por cada barrio.

In [ ]:
df_5barrios <- airbnb_entire[airbnb_entire$Barrio == "Recoletos" |
                           airbnb_entire$Barrio == "Goya" |
                           airbnb_entire$Barrio == "Sol" |
                           airbnb_entire$Barrio == "Trafalgar" |
                           airbnb_entire$Barrio == "Justicia"
                           ,c("Barrio","Precio", "MetrosCuadrados")]


In [ ]:
ggplot(df_5barrios, aes(x=Precio, color=Barrio)) + geom_density()


Calcula el tamaño medio, en metros cuadrados, para los 5 barrios anteriores y muestralo en el mismo dataframe junto con el precio y número de entradas

In [ ]:
# Los MetrosCuadrados = 0 se convierten a NA, para calcular el promedio correctamente
airbnb_entire_sinm0 <- airbnb_entire[!is.na(airbnb_entire$MetrosCuadrados) & airbnb_entire$MetrosCuadrados > 0 ,]

# DF con mean(MetrosCuadrados) por barrio
airbnb_m2_agg <- aggregate(MetrosCuadrados~Barrio, airbnb_entire_sinm0, mean, na.rm=TRUE)

# DF con Freq, Precio y mean(MetrosCuadrados) de Barrios con mas de 100 entradas
airbnb_pfm100 <- merge(x = airbnb_precio_freq_100, y = airbnb_m2_agg, by="Barrio", all.x=T)

# Ordena DF por precio
airbnb_pfm100 <- airbnb_pfm100[airbnb_pfm100$Precio %>% order(decreasing=T), ]

airbnb_pfm100 %>% head(5)


Dibuja el diagrama de densidad de distribución de los diferentes tamaños de apartamentos. Serían 5 gráficas, una por cada barrio.

In [ ]:
# Se eliminan las filas que contienen MetrosCuadrados = 0 y  MetrosCuadrados = NA
#df_5barrios_m <- df_5barrios[!is.na(df_5barrios$MetrosCuadrados) & df_5barrios$MetrosCuadrados > 0 ,]
df_5barrios_m <- df_5barrios[!is.na(df_5barrios$MetrosCuadrados),]

ggplot(df_5barrios_m, aes(x=MetrosCuadrados, color=Barrio))+geom_density()

Esta claro que las medias de metros cuadrados de cada uno de estos 5 barrios parecen ser diferentes, pero ¿son estadísticamente diferentes?
¿Que test habría que usar para comprobarlo?

In [ ]:
#df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Justicia"] %>% shapiro.test()

shapiro.test(df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Goya"])$p.value
shapiro.test(df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Justicia"])$p.value
shapiro.test(df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Recoletos"])$p.value
shapiro.test(df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Sol"])$p.value
shapiro.test(df_5barrios_m$MetrosCuadrados[df_5barrios_m$Barrio == "Trafalgar"])$p.value

In [ ]:
kruskal.test(MetrosCuadrados ~ Barrio, data = df_5barrios_m)

In [ ]:
# Como el p valor es relativamente alto, no podemos afirmar que el resultado sea
# estadisticamente significativo

Para únicamente los pisos de alquiler en el barrio de Sol:

``barrio_sol<-subset(airbnb_entire,Barrio=="Sol")``

Calcular un modelo lineal que combine alguna de estas variables:
* NumBanyos
* NumDormitorios
* MaxOcupantes
* MetrosCuadrados

In [ ]:
barrio_sol<-subset(airbnb_entire,Barrio=="Sol")

Primero calculamos la correlación para ver como se relacionan estas variables entre sí.

In [ ]:
cor(barrio_sol[,c("NumBanyos",
                  "NumDormitorios",
                  "MaxOcupantes",
                  "MetrosCuadrados")]
    , use="pairwise.complete.obs")

Se observa que la correlación entre el número de dormitorios y los metros cuadrados es sorprendentemente baja.
¿Son de fiar esos números?

Mediante un histograma o curvas de densidad podemos descartar números que notienen sentido en el dataframe barrio_sol,
para tener una matriz de correlación que tenga mayor sentido.


In [ ]:
ggplot(data=barrio_sol, aes(x=MetrosCuadrados))+
    geom_histogram(bins=40)+
    scale_x_continuous(breaks = seq(0, 400, by = 5))


In [ ]:
barrio_sol$MetrosCuadrados[barrio_sol$MetrosCuadrados < 20] <- NA

In [ ]:
ggplot(data=barrio_sol, aes(x=MetrosCuadrados))+
    geom_histogram(bins=40)+ 
    scale_x_continuous(breaks = seq(0, 400, by = 5))

In [ ]:
cor(barrio_sol[,c("NumBanyos",
                  "NumDormitorios",
                  "MaxOcupantes",
                  "MetrosCuadrados")]
    , use="pairwise.complete.obs")


Una vez que hayamos filtrado los datos correspondientes calcular el valor o la combinación de valores que mejor nos permite obtener el precio de un inmueble.

In [ ]:
model <- lm(data = barrio_sol, formula = Precio ~ NumBanyos + NumDormitorios + MaxOcupantes + MetrosCuadrados)
summary(model)

In [ ]:
# Como NumBanyos, MaxOcupantes, MetrosCuadrados tienen un Pr muy elevado por lo que no son representativos


In [ ]:
# model <- lm(data = barrio_sol, formula = Precio ~ NumDormitorios)
# summary(model)

¿Que variable es más fiable para conocer el precio de un inmueble, el número de habitaciones o los metros cuadrados?

In [ ]:
# El numero de habitaciones es mas fiable para conocer el precio del inmuble ya que su Pr es mucho menor"

Responde con su correspondiente margen de error del 95%, ¿cuantos euros incrementa el precio del alquiler por cada metro cuadrado extra del piso?

In [ ]:
model_m2 <- lm(data = barrio_sol, formula = Precio ~ MetrosCuadrados)
#summary(model_m2)
cnf_int_m2<-confint(model_m2)
cnf_int_m2

In [ ]:
# Por cada metro cuadrado extra, el precio de alquiler aumenta entre 0,68 y 1,30 

Responde con su correspondiente margen de error del 95%, ¿cuantos euros incrementa el precio del alquiler por cada habitación?

In [ ]:
model_h <- lm(data = barrio_sol, formula = Precio ~ NumDormitorios)
cnf_int_h<-confint(model_h)
cnf_int_h

In [ ]:
# Por cada habitacion, el precio de alquiler aumenta entre 30,5 y 36,22

¿Cual es la probabilidad de encontrar, en el barrio de Sol, un apartamento en alquiler con 3 dormitorios?
¿Cual es el intervalo de confianza de esa probabilidad?

In [ ]:
n_sol3hab <- barrio_sol[barrio_sol$NumDormitorios == 3,] %>% nrow()
n_sol <- barrio_sol %>% nrow()

binom.test(n_sol3hab, n_sol)


prob <- binom.test(n_sol3hab, n_sol)$estimate
conf_int <- binom.test(n_sol3hab, n_sol)$conf.int

paste("La probabilidad es de", prob, "con un intervalo de confianza del 95% de", conf_int[1], "y", conf_int[2])